In [2]:
import numpy as np
import pandas as pd
import matplotlib

import speech_recognition as sr
import soundfile
import spacy
import librosa
import glob
import os
from pydub import AudioSegment
from pydub.silence import split_on_silence
import pickle
import re
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score

In [3]:
file = Path.cwd() / 'nlp-hesitation/fillers.txt'
with open(file, 'r') as text:
    data = text.read()
    data = re.sub(r"(\/\*.*?\*\/)", "", data, flags=re.M|re.DOTALL)
#print(data.strip())
fillers = data.split("\n")

In [4]:
audio = Path.cwd() / 'nlp-hesitation/ted-audiofile.wav'
r = sr.Recognizer()
#chunks the current audio file, sr to translate audio chunk to text
def transcribe(path):
    sound = AudioSegment.from_wav(path)  
    chunk_list = split_on_silence(sound,
        min_silence_len = 900,
        # adjust this per requirement
        silence_thresh = sound.dBFS-14,
        # keep the silence for 1 second, adjustable as well
        keep_silence=700,
    )
    folder_name = "nlp-hesitation/audio-chunks"
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    for i, audio_chunk in enumerate(chunk_list, start=1):
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            try:
                text = r.recognize_google(audio_listened)
            except sr.UnknownValueError as e:
                print("Error:", str(e))
            else:
                text = f"{text.capitalize()}. "
                print(chunk_filename, ":", text)
                whole_text += text
    return whole_text

In [5]:
#print("\nFull text:", transcribe(str(audio)))
full_text = transcribe(str(audio))

Error: 
nlp-hesitation/audio-chunks/chunk2.wav : Today i'm going to like talk about talking. 
nlp-hesitation/audio-chunks/chunk3.wav : Last year in my elective class. 
nlp-hesitation/audio-chunks/chunk4.wav : We would like told that we were going to have like this big linguistics project. 
nlp-hesitation/audio-chunks/chunk5.wav : Since i didn't really know anything about linguistics apartments like how we speak. 
nlp-hesitation/audio-chunks/chunk6.wav : My mom is like always going on and on about how i use like too much. 
nlp-hesitation/audio-chunks/chunk7.wav : I thought that it would be like the perfect time to like look into it more. 
nlp-hesitation/audio-chunks/chunk8.wav : My project was centered on looking at the use of like as a filler word. 
nlp-hesitation/audio-chunks/chunk9.wav : In regards to gender age and education. 
nlp-hesitation/audio-chunks/chunk10.wav : Is my results can help me argue that just as i have used like several times since starting my talk. 
nlp-hesitation/

In [11]:
nlp = spacy.load('en_core_web_sm')
tokens = nlp(full_text)
for token in tokens:
    if token.pos_ == 'NOUN':
        print(token.text)

Today
year
class
linguistics
project
linguistics
apartments
mom
time
project
use
filler
word
regards
gender
age
education
results
times
talk
link
use
level
education
word
number
usages
language
project
use
filler
word
example
today
society
girls
girl
stereotype
teenager
clue
generalization
topic
teachers
linguistics
body
linguistics
bunch
texts
database
linguistics
body
data
technology
corpus
investigation
words
details
methodology
talk
hours
corpus
time
filler
word
information
females
males
age
group
correlation
level
education
investigation
guys
filler
word
comparison
verb
emotion
data
females
amount
males
furnace
type
females
account
number
words
genders
difference
use
number
words
gender
frequency
females
words
Females
males
difference


In [ ]:
#load RAVDESS dataset: https://zenodo.org/record/1188976
